In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import cv2
import pandas as pd
import numpy as np
import os

def draw_annotations(image_path, csv_path, output_path):
    # Read image
    image = cv2.imread(image_path)

    # Load annotation data from CSV file
    annotation_data = pd.read_csv(csv_path)

    # Iterate through each row in annotation data
    for index, row in annotation_data.iterrows():
        # Extract annotation information
        x, y, w, h = int(row['xc']), int(row['yc']), int(row['w']), int(row['h'])
        rotation_angle = float(row['Rotation_Angle(deg)'])
        track_number = int(row['Track'])

        # Calculate the coordinates of the four corners of the rotated bounding box
        center = (x, y)
        angle_rad = np.deg2rad(rotation_angle)
        dx = int(w / 2)
        dy = int(h / 2)
        rotated_rect_pts = cv2.boxPoints(((center), (w, h), rotation_angle))
        rotated_rect_pts = np.int0(rotated_rect_pts)

        # Draw rotated bounding box
        cv2.drawContours(image, [rotated_rect_pts], 0, (0, 255, 0), 2)

        # Draw rotation angle and track number
        cv2.putText(image, f"Angle: {rotation_angle}°", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(image, f"Track: {track_number}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the annotated image to the output path
    cv2.imwrite(output_path, image)

def process_folders(image_folder, annotation_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through each file in the image folder
    for image_filename in os.listdir(image_folder):
        if image_filename.endswith(".jpg") or image_filename.endswith(".png"):
            # Get the corresponding CSV file path
            csv_filename = os.path.splitext(image_filename)[0] + ".csv"
            csv_path = os.path.join(annotation_folder, csv_filename)

            # Check if the CSV file exists
            if os.path.exists(csv_path):
                # Get the image file path
                image_path = os.path.join(image_folder, image_filename)

                # Get the output file path
                output_filename = os.path.splitext(image_filename)[0] + "_annotated.jpg"
                output_path = os.path.join(output_folder, output_filename)

                # Call the function to draw annotations on the image
                draw_annotations(image_path, csv_path, output_path)
            else:
                print(f"CSV file not found for image: {image_filename}")

# Specify the input folder containing images
image_folder = "path/to/image/folder"

# Specify the input folder containing annotation CSV files
annotation_folder = "path/to/annotation/folder"

# Specify the output folder to save the annotated images
output_folder = "path/to/output/folder"

# Process the folders and save the annotated images
process_folders(image_folder, annotation_folder, output_folder)